In [1]:
import sys
import os
from os.path import dirname, realpath, join
base_dir = dirname(dirname(os.getcwd()))
import itertools
import pandas as pd
from os.path import join
base_dir

sys.path.insert(0, base_dir)
from config import PROSTATE_DATA_PATH, PLOTS_PATH, GENE_PATH, PROSTATE_LOG_PATH
from data_scripts.data_access import Data

In [2]:
from utilities.stats_utils_delong_xu import delong_roc_variance, delong_roc_test
from matplotlib import  pyplot as plt
from utilities.stats_utils import score_ci, pvalue, pvalue_stat
from sklearn import metrics
import numpy as np 

In [3]:
def read_predictions():
    all_models_dict = {}
    base_dir = PROSTATE_LOG_PATH
    models_base_dir = join(base_dir , 'other_ml_models/onesplit_ml_test')
    models = ['Linear Support Vector Machine ', 'RBF Support Vector Machine ', 'L2 Logistic Regression', 'Random Forest',
              'Adaptive Boosting', 'Decision Tree']

    for i, m in enumerate(models):
        df = pd.read_csv(join(models_base_dir, m + '_data_0_testing.csv'), sep=',', index_col=0, header=0)
        all_models_dict[m] = df

    pnet_base_dir = join(base_dir , 'prostate_net/onesplit_average_reg_10_tanh_test')
    df_pnet = pd.read_csv(join(pnet_base_dir, 'P-net_ALL_testing.csv'), sep=',', index_col=0, header=0)
    all_models_dict['P-net'] = df_pnet
    return all_models_dict

In [4]:
all_models_dict = read_predictions()

In [5]:
pnet_predictions = all_models_dict['P-net']
labels = pnet_predictions['y'].values.ravel()

In [6]:
all_models_dict['P-net'].head()

,pred,pred_scores,y
01-087MM_BONE,1.0,0.958771,1
01-095N1_LN,0.0,0.109515,1
08-093J1_LN,1.0,0.992522,1
10362,1.0,0.522076,0
AAPC-IP_LG-069-Tumor-SM-3NC72,0.0,0.220619,0


In [7]:
all_models_dict['Linear Support Vector Machine '].head()

,pred,pred_scores,y
01-087MM_BONE,1,0.789334,1
01-095N1_LN,0,0.235004,1
08-093J1_LN,1,0.967164,1
10362,1,0.831913,0
AAPC-IP_LG-069-Tumor-SM-3NC72,0,0.230290,0


In [8]:
score_fun={}
score_fun['Accuracy'] = metrics.accuracy_score
score_fun['Precision'] = metrics.precision_score
score_fun['AUC'] = metrics.roc_auc_score
score_fun['F1'] = metrics.f1_score
score_fun['AUPR'] = metrics.average_precision_score
score_fun['Recall'] = metrics.recall_score

In [9]:
def fdr(p_vals):
#https://stackoverflow.com/questions/25185205/calculating-adjusted-p-values-in-python
    from scipy.stats import rankdata
    ranked_p_values = rankdata(p_vals)
    fdr = p_vals * len(p_vals) / ranked_p_values
    fdr[fdr > 1] = 1

    return fdr

In [10]:
results=[]
pvalue_list=[]
delong_results=[]
for i, (model_name, predictions_df) in enumerate(all_models_dict.items()):
    if model_name!='P-net':
        pred2 = predictions_df['pred_scores'].values.ravel()
        for func_name, func  in score_fun.items():
                
            if func_name in ['AUC', 'AUPR']:
                col_name= 'pred_scores'
            else:
                col_name= 'pred'
                
            pred_pnet= pnet_predictions[col_name].values.ravel()
            pred_model= predictions_df[col_name].values.ravel()
            
            if func_name=='AUC':
                pvalue_ = delong_roc_test(labels, pred_pnet, pred_model)
                pvalue_delong = 10**pvalue_[0][0]/2
                delong_results.append({'measure': 'AUC_DeLong', 'model':model_name, 'pvalue': pvalue_delong}) 
            
            stat_fun= np.median
            p, scores1, scores2, z = pvalue_stat( labels,pred_pnet,pred_model, func,n_bootstraps=2000, two_tailed=False, seed=1234, stat_fun=np.median)
            med_pnet = stat_fun(scores1)
            med_model = stat_fun(scores2)
            stat_fun_diff =  med_pnet - med_model
            results.append({'measure': func_name, 'model':model_name, 'pvalue': p, 'model median': med_model, 'P-NET median':med_pnet, 'Median difference': stat_fun_diff }) 

In [11]:
results_df = pd.DataFrame(results)

In [12]:
results_df['pvalue_fdr_adjusted']= results_df.groupby('measure')['pvalue'].apply(fdr)

In [13]:
results_df = results_df.set_index(['model', 'measure'])
results_df = results_df.round(3)

In [14]:
results_df

pvalue  model median  P-NET median  \
model                          measure                                         
Linear Support Vector Machine  Accuracy    0.334         0.824         0.838   
                               Precision   0.942         0.807         0.725   
                               AUC         0.134         0.907         0.927   
                               F1          0.041         0.695         0.768   
                               AUPR        0.123         0.857         0.887   
                               Recall      0.000         0.612         0.823   
RBF Support Vector Machine     Accuracy    0.392         0.828         0.838   
                               Precision   0.977         0.824         0.725   
                               AUC         0.236         0.915         0.927   
                               F1          0.058         0.702         0.768   
                               AUPR        0.138         0.860         0.887   
                               Recall      0.000         0.612         0.823   
L2 Logistic Regression         Accuracy    0.452         0.833         0.838   
                               Precision   0.975         0.816         0.725   
                               AUC         0.120         0.908         0.927   
                               F1          0.128         0.719         0.768   
                               AUPR        0.020         0.846         0.887   
                               Recall      0.000         0.643         0.823   
Random Forest                  Accuracy    0.082         0.794         0.838   
                               Precision   0.646         0.745         0.725   
                               AUC         0.011         0.877         0.927   
                               F1          0.006         0.642         0.768   
                               AUPR        0.001         0.788         0.887   
                               Recall      0.000         0.564         0.823   
Adaptive Boosting              Accuracy    0.591         0.843         0.838   
                               Precision   1.000         0.930         0.725   
                               AUC         0.016         0.889         0.927   
                               F1          0.092         0.705         0.768   
                               AUPR        0.021         0.831         0.887   
                               Recall      0.000         0.568         0.823   
Decision Tree                  Accuracy    0.657         0.848         0.838   
                               Precision   0.992         0.875         0.725   
                               AUC         0.000         0.830         0.927   
                               F1          0.202         0.730         0.768   
                               AUPR        0.000         0.733         0.887   
                               Recall      0.000         0.627         0.823   

                                          Median difference  \
model                          measure                        
Linear Support Vector Machine  Accuracy               0.015   
                               Precision             -0.082   
                               AUC                    0.020   
                               F1                     0.073   
                               AUPR                   0.029   
                               Recall                 0.211   
RBF Support Vector Machine     Accuracy               0.010   
                               Precision             -0.099   
                               AUC                    0.013   
                               F1                     0.066   
                               AUPR                   0.027   
                               Recall                 0.211   
L2 Logistic Regression         Accuracy               0.005   
                               Precision             -0.

In [15]:
#results_df.to_csv('model_comparison_pvalue.csv')

In [16]:
results_delong_df = pd.DataFrame(delong_results)
results_delong_df['pvalue_fdr_adjusted']= results_delong_df.groupby('measure')['pvalue'].apply(fdr)
results_delong_df = results_delong_df.set_index(['model', 'measure'])
results_delong_df=results_delong_df.round(3)
#results_delong_df.to_csv('model_comparison_delong_pvalue.csv')
results_delong_df

,,pvalue,pvalue_fdr_adjusted
model,measure,,
Linear Support Vector Machine,AUC_DeLong,0.126,0.151
RBF Support Vector Machine,AUC_DeLong,0.246,0.246
L2 Logistic Regression,AUC_DeLong,0.122,0.184
Random Forest,AUC_DeLong,0.013,0.038
Adaptive Boosting,AUC_DeLong,0.023,0.046
Decision Tree,AUC_DeLong,0.001,0.004


In [17]:
df = pd.DataFrame(results)
df_cross = pd.crosstab( df['model'], df['measure'], values=df.pvalue, aggfunc='first') 

df_cross

measure,AUC,AUPR,Accuracy,F1,Precision,Recall
model,,,,,,
Adaptive Boosting,0.0165,0.0205,0.5910,0.0925,0.9995,0.0000
Decision Tree,0.0000,0.0000,0.6570,0.2015,0.9915,0.0000
L2 Logistic Regression,0.1200,0.0200,0.4525,0.1280,0.9750,0.0005
Linear Support Vector Machine,0.1340,0.1230,0.3335,0.0410,0.9420,0.0000
RBF Support Vector Machine,0.2360,0.1375,0.3925,0.0575,0.9770,0.0000
Random Forest,0.0110,0.0010,0.0825,0.0060,0.6455,0.0000


In [18]:
df_cross_fdr_adjusted = df_cross.copy()
for c in df_cross_fdr_adjusted.columns:
    df_cross_fdr_adjusted[c] = fdr(df_cross_fdr_adjusted[c])

In [19]:
# df_cross_fdr_adjusted.applymap('{:.3f}'.format)
df_cross_fdr_adjusted = df_cross_fdr_adjusted.round(3)
df_cross_fdr_adjusted

measure,AUC,AUPR,Accuracy,F1,Precision,Recall
model,,,,,,
Adaptive Boosting,0.033,0.031,0.709,0.139,1.0,0.0
Decision Tree,0.000,0.000,0.657,0.202,1.0,0.0
L2 Logistic Regression,0.180,0.040,0.679,0.154,1.0,0.0
Linear Support Vector Machine,0.161,0.148,1.000,0.123,1.0,0.0
RBF Support Vector Machine,0.236,0.138,0.785,0.115,1.0,0.0
Random Forest,0.033,0.003,0.495,0.036,1.0,0.0


In [20]:
#df_cross_fdr_adjusted.to_csv('df_cross_fdr_adjusted.csv')